In [13]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import warnings


warnings.filterwarnings("ignore", category=DeprecationWarning)

# Loading the trained wrinkle detection model
model = tf.keras.models.load_model('wrinkle_detection_model.h5')

class WrinkleDetectorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Wrinkle Detector App")

        self.create_widgets()

    def create_widgets(self):
        self.label = tk.Label(self.root, text="Wrinkle Detector App", font=("Helvetica", 16))
        self.label.pack(pady=10)

        self.upload_button = tk.Button(self.root, text="Upload Image", command=self.upload_image)
        self.upload_button.pack(pady=10)

        self.image_label = tk.Label(self.root)
        self.image_label.pack(pady=10)

        self.predict_button = tk.Button(self.root, text="Predict Wrinkles", command=self.predict_wrinkles)
        self.predict_button.pack(pady=10)

        self.result_label = tk.Label(self.root, text="")
        self.result_label.pack(pady=10)

    def upload_image(self):
        file_path = filedialog.askopenfilename(title="Select Image", filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
        if file_path:
            self.display_image(file_path)

    def display_image(self, file_path):
        img = Image.open(file_path)

        # Converting the image to RGBA format if it has a palette and transparency
        if img.mode == 'P' and 'transparency' in img.info:
            img = img.convert('RGBA')

        img = img.resize((300, 300), Image.LANCZOS)
        img = ImageTk.PhotoImage(img)

        self.image_label.configure(image=img)
        self.image_label.image = img

        self.result_label.config(text="")  

        self.image_path = file_path

    def predict_wrinkles(self):
        if hasattr(self, 'image_path'):
            result = self.predict_wrinkles_from_image(self.image_path)
            self.result_label.config(text=result)
        else:
            self.result_label.config(text="Please upload an image first.")

    def predict_wrinkles_from_image(self, image_path):
        img = image.load_img(image_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0  

        prediction = model.predict(img_array)

        # Class 1 corresponds to wrinkles, while Class 0 corresponds to no wrinkles
        if prediction[0, 0] >= 0.5:
            return "The person has wrinkles on their face."
        else:
            return "The person does not have wrinkles on their face."

if __name__ == "__main__":
    root = tk.Tk()
    app = WrinkleDetectorApp(root)
    root.mainloop()


1/1 [==============================] - 0s 132ms/step
